In [1]:
import sys, os
print("Python exe:", sys.executable)
print("CWD:", os.getcwd())

Python exe: /Users/nickhellmer/vibedrop/venv/bin/python
CWD: /Users/nickhellmer/vibedrop


In [25]:
sql_df("""
SELECT tc.table_name, kcu.column_name, ccu.table_name AS target_table
FROM information_schema.table_constraints AS tc
JOIN information_schema.key_column_usage AS kcu
  ON tc.constraint_name = kcu.constraint_name
JOIN information_schema.constraint_column_usage AS ccu
  ON ccu.constraint_name = tc.constraint_name
WHERE tc.constraint_type = 'FOREIGN KEY'
ORDER BY 1,2;
""")

,table_name,column_name,target_table
0,circle_memberships,circle_id,sound_circles
1,circle_memberships,user_id,users
2,song_feedback,song_id,submissions
3,song_feedback,user_id,users
4,sound_circles,creator_id,users
5,submissions,circle_id,sound_circles
6,submissions,user_id,users
7,vibe_scores,user1_id,users
8,vibe_scores,user2_id,users


In [21]:
sql_df("""
SELECT column_name FROM information_schema.columns
WHERE table_name='song_feedback' ORDER BY 1;
""")

,column_name
0,feedback
1,id
2,song_id
3,timestamp
4,user_id


# Connect + List Table Names

In [2]:
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine, text
import pandas as pd

load_dotenv()  # reads DATABASE_URL if you set it; falls back to local
DB_URL = os.getenv("DATABASE_URL", "postgresql://localhost/vibedrop_dev")
engine = create_engine(DB_URL, future=True)

def sql_df(q: str, **params) -> pd.DataFrame:
    with engine.begin() as conn:
        return pd.read_sql(text(q), conn, params=params)

# See what tables you’ve got:
sql_df("""
SELECT table_name 
FROM information_schema.tables 
WHERE table_schema='public'
ORDER BY table_name
""")

,table_name
0,alembic_version
1,circle_memberships
2,song_feedback
3,sound_circles
4,submissions
5,users
6,vibe_scores


# **View all Tables**

In [19]:
sql_df("""
SELECT * FROM "user"
""")

,id,spotify_id,vibedrop_username,display_name,email,access_token,refresh_token,expires_at,drop_cred,created_at
0,1,7xw4yczo4i8q0fjnd2ytyu5fd,nickhellmer,None,None,BQAKzmJ-uVI-4cDpnWAJburHeLQHI28LIGmOyAHSnyUFyl...,AQAHczF0WpCtsGZqmGdESxe4yq6KsOEGHiEqRKjpj8lUvF...,2025-08-08 22:11:19.383166,5.0,2025-08-07 14:12:18.625936


In [11]:
sql_df("""
SELECT * FROM circle_membership
""")

,id,user_id,circle_id,joined_at
0,1,1,1,2025-08-07 14:49:47.245475
1,2,1,2,2025-08-07 15:26:35.323571
2,3,1,3,2025-08-07 15:47:51.044401
3,4,1,4,2025-08-07 15:48:26.602886
4,5,1,5,2025-08-08 10:07:15.219839
5,6,1,6,2025-08-08 20:57:52.337975
6,7,1,7,2025-08-08 20:59:30.307543
7,8,1,8,2025-08-10 18:59:53.296348


In [12]:
sql_df("""
SELECT * FROM sound_circle
""")

,id,circle_name,drop_frequency,drop_day1,drop_day2,drop_time,invite_code,creator_id,created_at
0,1,LOCAL test 1,Daily,Monday,Monday,2025-08-07 15:00:00,ASMDRJWC,1,2025-08-07 14:49:47.232383
1,2,LOCAL test 2 5pm EST,Daily,Monday,Monday,2025-08-07 16:00:00,b5TSqMMr,1,2025-08-07 15:26:35.317472
2,3,LOCAL test 3 6pm EST,Daily,Monday,Monday,2025-08-07 17:00:00,Kh3voc7w,1,2025-08-07 15:47:51.031482
3,4,LOCAL test 4 7pm EST,Daily,Monday,Monday,2025-08-07 18:00:00,4m8z0NER,1,2025-08-07 15:48:26.598587
4,5,LOCAL biweekly test,Biweekly,Friday,Saturday,2025-08-08 10:00:00,rnYBauJd,1,2025-08-08 10:07:15.207261
5,6,LOCAL Test Weekly 10pm EST,Weekly,Friday,Monday,2025-08-08 21:00:00,wDeUm05R,1,2025-08-08 20:57:52.328039
6,7,LOCAL Test Weekly 11pm EST,Weekly,Friday,Monday,2025-08-08 22:00:00,rOf51C3r,1,2025-08-08 20:59:30.301429
7,8,LOCAL 9PM DIALY,Daily,Monday,Monday,2025-08-10 20:00:00,K4UDlxyi,1,2025-08-10 18:59:53.280287


In [13]:
sql_df("""
SELECT * FROM submission
""")

,id,circle_id,user_id,spotify_track_id,cycle_date,submitted_at,visible_to_others
0,1,1,1,4Zbsau2HqGebCOrUxQJzEl,2025-08-07,2025-08-07 19:51:08.822823+00:00,False
1,2,2,1,3PpxUVqvRIkVV7q5jWiQK9,2025-08-07,2025-08-07 20:26:55.285818+00:00,False
2,3,3,1,4YS0MW1khH9F0Ktg4QgMZN,2025-08-07,2025-08-07 20:47:58.203684+00:00,False
3,4,4,1,7H3bWxgjLiSnwStlBGgAlG,2025-08-07,2025-08-07 20:48:48.189816+00:00,False
4,5,2,1,30l8jqVysaGj7bPcGSVUgD,2025-08-07,2025-08-08 01:58:28.141362+00:00,False
5,6,5,1,7H3bWxgjLiSnwStlBGgAlG,2025-08-08,2025-08-08 15:07:39.266722+00:00,False
6,7,4,1,6uToph829N0d0VyfouEYer,2025-08-07,2025-08-08 19:50:02.029746+00:00,False
7,8,6,1,3PczYtUqSoEXhXQlzbMDIG,2025-08-08,2025-08-09 01:58:48.842896+00:00,False
8,9,7,1,4OjV2BCKlDBmUakKOARRqt,2025-08-01,2025-08-09 01:59:46.189679+00:00,False
9,10,2,1,1gPeJMO7Az6ZtzHiaKUTWb,2025-08-10,2025-08-10 21:46:43.440035+00:00,False


In [14]:
sql_df("""
SELECT * FROM vibe_score
""")

,id,user1_id,user2_id,vibe_index,last_updated


In [15]:
sql_df("""
SELECT * FROM alembic_version
""")

,version_num
0,0695c42d37d7


# **Feedback Queries**

### *Full Feedback Table*

In [4]:
sql_df("""
SELECT * FROM song_feedback
""")

,id,user_id,song_id,feedback,timestamp
0,1,1,9,dislike,2025-08-11 20:44:12.697622


### *Feedback for a circle by ID*

In [18]:
circle_id = 7 # change me

df = sql_df("""
SELECT
  f.id,
  f.user_id,
  f.feedback,
  f.timestamp,
  s.id,
  s.spotify_track_id,
  s.user_id      AS submitter_id,
  s.circle_id
FROM song_feedback f
JOIN submission s ON f.song_id = s.id
WHERE s.circle_id = :cid
ORDER BY f.timestamp DESC
""", cid=circle_id)
df

,id,user_id,feedback,timestamp,id,spotify_track_id,submitter_id,circle_id
0,1,1,like,2025-08-11 16:43:32.678338,9,4OjV2BCKlDBmUakKOARRqt,1,7


In [17]:
df2 = sql_df("""
SELECT
  f.id           AS feedback_id,
  f.feedback     AS feedback_value,
  f.timestamp    AS feedback_at,
  rater.vibedrop_username   AS rater_username,
  submitter.vibedrop_username AS submitter_username,
  s.spotify_track_id,
  s.circle_id
FROM song_feedback f
JOIN submission s         ON f.song_id = s.id
JOIN "user" rater         ON rater.id = f.user_id
JOIN "user" submitter     ON submitter.id = s.user_id
WHERE s.circle_id = :cid
ORDER BY f.timestamp DESC
""", cid=circle_id)
df2

,feedback_id,feedback_value,feedback_at,rater_username,submitter_username,spotify_track_id,circle_id
0,1,like,2025-08-11 16:43:32.678338,nickhellmer,nickhellmer,4OjV2BCKlDBmUakKOARRqt,7


# New Section

# New Section

# New Section

# New Section

# New Section